In [1]:
# default_exp core

# covid

> API details.

In [2]:
#hide
from nbdev.showdoc import *

TBD: Add documentation

In [3]:
#export
import typing
from typing import List, Callable
import pandas as pd
import requests
import datetime
from datetime import date
import io
import os
from io import StringIO
import urllib.request
import matplotlib.pyplot as plt

In [4]:
#export
# Note that the name of these files changed overnight on 24.03.20
CONFIRMED = 'time_series_covid19_confirmed_global.csv'
DEATHS    = 'time_series_covid19_deaths_global.csv'
RECOVERED = 'time_series_19-covid-Recovered.csv'

In [5]:
#export
getToday: Callable[[None], str] = lambda: date.today().strftime('%m-%d-%Y')
getYesterday: Callable[[None], str] = lambda: (date.today() - datetime.timedelta(days = 1)).strftime('%m-%d-%Y')

In [6]:
#export
def procDataframe(csv: str) -> pd.DataFrame:
    df = pd.read_csv(csv)
    try:
        df['Province/State'].fillna('',inplace=True)
    except:
        df['Province_State'].fillna('',inplace=True)
    df.fillna(0,inplace=True)
    cols = df.columns.to_list()
    if 'Last Update' in cols:
        df['Last Update'] = df['Last Update'].apply(pd.to_datetime)
    if 'Last_Update' in cols:
        df['Last_Update'] = df['Last_Update'].apply(pd.to_datetime)
    return df

def procUrl(url: str,download: bool,localfile: str=None,force: bool=False, verbose: bool=False) -> pd.DataFrame:
    if download:
        if os.path.exists(localfile) and not force:
            verbose and print(f'"{localfile}" already exists so will not overwrite')
        else:
            verbose and print(f'Downloading "{localfile}" from "{url}"...')
            urllib.request.urlretrieve(url,localfile)
        return procDataframe(localfile)
    else:
        s = requests.get(url).content
        return procDataframe(io.StringIO(s.decode('utf-8')))

In [7]:
#export
def getTimeSeriesConfirmed(download: bool=False,force: bool=False) -> pd.DataFrame:
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{CONFIRMED}'
    localfile = 'time_series_19-covid-Confirmed.csv'
    return procUrl(url,download,localfile,force)

def getTimeSeriesDeaths(download: bool=False,force: bool=False) -> pd.DataFrame:
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{DEATHS}'
    localfile = 'time_series_19-covid-Deaths.csv'
    return procUrl(url,download,localfile,force)

def getTimeSeriesRecovered(download: bool=False,force: bool=False) -> pd.DataFrame:
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{RECOVERED}'
    localfile = 'time_series_19-covid-Recovered.csv'
    return procUrl(url,download,localfile,force)

def getDailyReport(day: str,download: bool=False,force: bool=False) -> pd.DataFrame:
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{day}.csv'
    localfile = f'{day}.csv'
    return procUrl(url,download,localfile,force)

In [8]:
#export
def procTimeSeriesDataframe(r: List) -> pd.DataFrame:
    sdf = pd.DataFrame(r)
    sdf['day'] = sdf['day'].apply(pd.to_datetime)
    sdf.set_index('day',drop=True,inplace=True)
    return sdf

In [9]:
#export
def procTimeSeries(df: pd.DataFrame) -> pd.DataFrame:
    r = []
    countries = df.groupby('Country/Region')
    cols = df.columns.to_list()
    for country,group in countries:
        total = []
        for row_index, row in group.iterrows():
            rvals = row.to_list()
            if not len(total):
                total = rvals[4:]
                #print('first',total)
            else:
                #print('next',rvals[4:])
                total = [a+b for a,b in zip(total,rvals[4:])]
        for a, b in zip(cols[4:], total):
            r.append({'day':a, 'country':country, 'confirmed':b})
    return procTimeSeriesDataframe(r)

In [10]:
#export
procTimeSeriesDeaths: Callable[[pd.DataFrame], pd.DataFrame] = lambda: procTimeSeries(getTimeSeriesDeaths(download=True))
procTimeSeriesConfirmed: Callable[[pd.DataFrame], pd.DataFrame] = lambda: procTimeSeries(getTimeSeriesConfirmed(download=True,force=True))
procTimeSeriesRecovered: Callable[[pd.DataFrame], pd.DataFrame] = lambda: procTimeSeries(getTimeSeriesRecovered(download=True))

In [11]:
#export
def plotCountryTimeSeries(df: pd.DataFrame,countries: List,kind: str) -> None:
    fig, ax = plt.subplots()
    for country in countries:
        ax = df[df['country'] == country].plot(ax=ax,kind='line',figsize=(18,9))
    ax.set_ylabel('Count', size=14)
    ax.set_xlabel('Day', size=14)
    ax.set_title(f'{kind} in {countries} as of {getYesterday()}',size=18)
    ax.legend(ax.get_lines(),countries)
    plt.show()

In [12]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
